# add c_EC as one of the inputs for diffusivity and conductivity

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    sigma_e = (
        (c_EC <= 4541) * (x/4.541) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)) 
        + (c_EC > 4541) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000))
    )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    D_c_e = (
        (c_EC <= 4541) * (1/4.541*x) * (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        +  (c_EC > 4541) *  (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10)
    )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 197 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =    -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


In [7]:
Sim_Landes_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_s,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sim_Landes_sdiff    = pybamm.Simulation(
    Model_sdiff, experiment = Experiment_s,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)  
Sol_Landes_ddiff    = Sim_Landes_ddiff.solve();
Sol_Landes_sdiff    = Sim_Landes_sdiff.solve();

In [8]:
Sol_Landes_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,-1] 

array([ 0.62042432,  0.62487915,  0.63367444,  0.64656455,  0.6631529 ,
        0.68288925,  0.70509606,  0.72902056,  0.75389625,  0.77899646,
        0.80367003,  0.82735874,  0.84960115,  0.87002903,  0.88836065,
        0.90439377,  0.91799918,  0.92911495,  0.9377414 ,  0.94393646,
        0.94628923,  0.94649196,  0.94668728,  0.94687527,  0.94705601,
        0.94722956,  0.94739601,  0.92007536,  0.9032094 ,  0.88612317,
        0.86879853,  0.85121557,  0.83335232,  0.81518446,  0.7966849 ,
        0.7778233 ,  0.75856549,  0.73887268,  0.71870048,  0.69799775,
        0.57673419,  0.39808108,  0.25216775,  0.13419435,  0.04257241,
       -0.00121636,  0.00413054,  0.01769375,  0.01921146,  0.02631594,
        0.03427867,  0.0421593 ,  0.04890609,  0.05378479,  0.05647384,
        0.05703538,  0.05584807,  0.05353785,  0.05092509,  0.04900693])

In [12]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [13]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
quick_plot.create_gif(
    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as linear.gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.05472222222222222, step=0.0005472222222222…

In [14]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
quick_plot.create_gif(
    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as linear.gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.05472222222222222, step=0.0005472222222222…